In [1]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
from distributed import LocalCluster, Client

In [2]:
cluster = LocalCluster(n_workers=2, threads_per_worker=4)#silence_logs=logging.ERROR)
client = Client(cluster)
client

/home/alewis/miniconda3/envs/scipy2020/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37203 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:44355 Dashboard: http://127.0.0.1:37203/status,Cluster Workers: 2 Cores: 8 Memory: 67.47 GB


In [3]:
# allow many retries
dask.config.config['distributed']['scheduler']['allowed-failures'] = 10

In [4]:
df = dd.read_parquet('../data/resaved-gps-points', engine='pyarrow')
# df = df.partitions[45:46].loc[:1_000_000, :].repartition(npartitions=8).persist()
# df.head()

In [ ]:
%%time 
t0 = time.time()
dfs = []

with open(f'{datetime.now()}_log.txt', 'w') as f:
    for i in range(300, 600):
        try:
            temp_df = df.partitions[i]
            temp_df['geohash'] = temp_df.apply(lambda row: geohash.encode(row.latitude, row.longitude), axis=1, meta='object')
            temp_df.to_parquet(f'../data/geohashed_gps_points3/{i:04d}')
        except Exception as e:
            msg = str(e) + f'\n partition {i:04d} \n ---'
            print(msg)
            f.write(msg)
dt_hr = (time.time() - t0)/60/60